In [1]:
#%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pickle
import json
from scipy.stats import wilcoxon
from scipy.optimize import least_squares
import sys
from data_tools import load_data

MODELFILE = 'test_pds.p'

In [2]:
# load estimated score matrix
data = pickle.load(open(MODELFILE, 'rb'))
est_scores = data['scores']

In [3]:
# load true score matrix
tru_scores, labels = load_data('sim.mat', feature_list=['scores'])
tru_scores = np.squeeze(np.asarray(tru_scores))

In [4]:
from itertools import permutations

def scale_scores(caus_scores, tru_scores):
    f_perm = list(permutations([0, 1, 2]))

    best_err = np.inf
    for k, p in enumerate(f_perm):
        these_scores = caus_scores[:,p]
        caus_resid = lambda scaling : (these_scores*scaling - tru_scores).flat
        res = least_squares(caus_resid, np.ones(3))
        if res.cost < best_err:
            best_err = res.cost
            caus_scale = res.x
            best_perm = p

    caus_est = caus_scores[:,best_perm]*caus_scale
    return caus_est    
    
scaled_est = scale_scores(est_scores, tru_scores)

In [5]:
# pairwise comparison between absolute residuals
from scipy.stats import spearmanr, norm
from math import tanh, atanh

def spear_CI(r2, n, alpha=0.05):
    z = norm().ppf(1 - alpha/2)
    st_err = np.sqrt((1+r2/2)/(n-3))
    low = tanh(atanh(r2) - z*st_err)
    up = tanh(atanh(r2) + z*st_err)
    return (low, up)
    
for N in range(3):
    this_est = scaled_est[:,N]
    s, _ = spearmanr(this_est, tru_scores[:,N])
    l, u = spear_CI(s, len(this_est))
    
    print(f"Factor {N+1:d} - R2={s:.5f} low={l:.5f} high={u:.5f}")

Factor 1 - R2=0.96197 low=0.92516 high=0.98085
Factor 2 - R2=0.94986 low=0.90206 high=0.97464
Factor 3 - R2=0.89731 low=0.80552 high=0.94705
